In [1]:
import re
import json

In [2]:
def get_real(value):
    
    value_str = str(value)
    
    if value_str == "None":
        return None
    
    try:
        return int(value_str)
    except:
        pass

    try:
        return float(value_str)
    except:
        pass

    try:
        if value_str.lower() == "true":
            return True
        elif value_str.lower() == "false":
            return False
    except:
        pass

    if value_str.startswith("'") and value_str.endswith("'"):
        return value_str[1:-1]

    return value

In [3]:
def get_params(raw_params):
    # raw_params = ["1", "2.36", "x", "y", "a =1", "  b= '2'", "c=true  ", "    d =  'z'        ", "e=1.589"]
    # return:
    # (args, kwargs)
    # args = [1,2.36,"x","y"]
    # kwargs = {"a":1, "b": '2', "c": True, "d": 'z', "e": 1.589}
    assert isinstance(raw_params, (list, tuple)), "parameter `raw_params` shoule be a list or tuple"
    args, kwargs = [], {}
    for raw_param in raw_params:
        formated_raw_param = re.sub(r"\s*=\s*", "=", raw_param.strip())
        kwarg_matched = re.search(r"\s*(.*)\s*=\s*(.*)\s*", formated_raw_param)
        if kwarg_matched:
            key, value = kwarg_matched.groups()
            # value can be int, float, str, bool
            kwargs[key] = get_real(value)
        else:
            args.append(get_real(raw_param))
    return args, kwargs

In [4]:
def clause_string_to_dict(clause):
    clause = clause.strip()
    matched = re.search(r'(["\d\']\s*\b(and|AND|img)\b\s*)', clause)
    if matched:
        _clause = re.sub(r'["d\']\s*(?:\band\b)\s*', '",', clause).split('",')
#         print(_coditions)
        clause_list = [_w + '"' for _w in _clause[:-1]]
        clause_list.append(_clause[-1])
        print(clause_list)
        return get_params(clause_list)
#         return ",".join(list(map(lambda x: x[0:x.find('=')], formated_clause)))
    else:
        return {}

In [5]:
normal_where = 'type="Nurses Station" and description="meow"'
crazy_where = ' type="护士站主机  "  and `descr an\'d iption`="a,b"  '
print(json.dumps(clause_string_to_dict(normal_where)[1], indent=4))
print(json.dumps(clause_string_to_dict(crazy_where)[1], indent=4))

['type="Nurses Station"', 'description="meow"']
{
    "description": "\"meow\"",
    "type": "\"Nurses Station\""
}
['type="护士站主机  "', '`descr an\'d iption`="a,b"']
{
    "type": "\"\u62a4\u58eb\u7ad9\u4e3b\u673a  \"",
    "`descr an'd iption`": "\"a,b\""
}
